## Simulate Ultrasound Image from CT slice example

In [2]:
import cv2
import time
import torch
import numpy as np
import torch.nn.functional as F

from unet.model import UNet_dilation, UNet, AttentionUNet
from utils.vis import array2tensor, tensor2array

### Make sure GPU is available

In [4]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device {device}")

Using device cpu


### load pretrained network

In [8]:
net = UNet_dilation(n_channels=1, n_classes=1, bilinear=False)
net.to(device=device)
net.load_state_dict(torch.load('model/ct2us_unet_dilation_cur.pth'))
print(net.eval())

RuntimeError: Attempting to deserialize object on a CUDA device but torch.cuda.is_available() is False. If you are running on a CPU-only machine, please use torch.load with map_location=torch.device('cpu') to map your storages to the CPU.

### Load example CT slice 

In [7]:
image = cv2.imread('../dataset/ct_slices_trans_06-Mar-2024/ct_msk/ctmsk_100.png', cv2.IMREAD_GRAYSCALE)

### Simulate Ultrasound from CT slice

In [ ]:
start = time.perf_counter()

image = array2tensor(image, device=device)
print(f'input shape: {image.shape}')

pred = net(image)
us_sim = tensor2array(pred)*255

print(f'prediction shape: {pred.shape}')
print(f'time elapsed: {(time.perf_counter()-start):.3f} sec')  # benchmarking

cv2.imwrite('test_us_sim_out.png', us_sim)